### Download the Dependencies

In [ ]:
import pandas as pd
import numpy as np

### Loading the Data

In [ ]:
df= pd.read_excel('IT Incidents Dashboard FY22 Q4 V1.xlsx')

### Showing the Data

In [ ]:
df.head()

### Datatype of Features

In [ ]:
df.info()

### Checking the Missing Values

In [ ]:
df.isnull().sum()

### Considering the tickets of Closed States

In [ ]:
df =df[df['Incident state']=='Closed']

### Dropping the irreleavant Columns

In [ ]:
df.drop(['YY-MM','Assigned to','Customer','Updated','Description','Active','RFC','Caused by Change',
         'Resolution Action','Explain Resolution','Resolve time','SLA due','Due date','Urgency',
         'Actual end date','Actual start date','Resolution Code','Contact Type','Resolved by','Subcategory','Category','Made SLA'],axis=1,inplace=True)

In [ ]:
df.columns

In [ ]:
df.head()

#### After dropping the columns, checking the misssing values of Dataframe

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df['Business resolve time'] = df['Business resolve time'].fillna(0)

In [ ]:
df.isnull().sum()

In [ ]:
df[df['Business resolve time']==0]

In [ ]:
### Dropping the misssing values from specified columns

In [ ]:
df = df.dropna(subset=['Name','Resolved'])

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.dtypes

#### There are some datatime Data in Business Resolve time column.so Not considering those Data in DataFrame

In [ ]:
df=df.loc[(df['Business resolve time'] !='1-Sep-06')&(df['Business resolve time'] !='20-Jun-13')&(df['Business resolve time'] !='16-Feb-55')
      &(df['Business resolve time'] !='11-Aug-52')&(df['Business resolve time'] !='28-Dec-86')&(df['Business resolve time'] !='20-Jul-36')&
      (df['Business resolve time'] !='20-Jul-36')&(df['Business resolve time'] !='24-Aug-60')&(df['Business resolve time'] !='27-Jan-26')&
      (df['Business resolve time'] !='27-Sep-41')&(df['Business resolve time'] !='23-Oct-30')&(df['Business resolve time'] !='20-Jul-36')&
     (df['Business resolve time'] !='12-Jun-58')&(df['Business resolve time']!='23-Aug-84')&(df['Business resolve time']!='24-Jan-54')]

In [ ]:
df.dtypes

In [ ]:
df.shape

#### Changing the data types for Resolved,Opened and Business resolve time columns

In [ ]:
df['Business resolve time']=pd.to_numeric(df['Business resolve time'],errors='coerce')

In [ ]:
df['Resolved']=pd.to_datetime(df['Resolved'])
df['Opened']=pd.to_datetime(df['Opened'])

In [ ]:
df.dtypes

In [ ]:
#therer are lots of negative values also
# We will take absolute values 
 
df['seconds_to_complete']=np.abs(np.array((df['Resolved']-df['Opened'])))

In [ ]:
df['seconds_to_complete']=df['seconds_to_complete']/np.timedelta64(1,'s')

In [ ]:
df.head()

In [ ]:
df['Business resolve time']=df['Business resolve time'].mask(df['Business resolve time'].eq(0.0),df['seconds_to_complete'])


# df[(df['Business resolve time']==0)|(df['Business resolve time'] =='')]
# df.loc[(df['Business resolve time']==0)|(df['Business resolve time'] ==''),'result']='success'

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df.to_csv('check.csv',index=False)

In [ ]:
df.dtypes

#### Saving the Preprocessed Data in csv

In [ ]:
df.to_csv('Preprocessed_data.csv',index=False)

#### How many assignment groups are present in Preprocessed Data

In [ ]:
assignment_group_list = df['Assignment group'].unique().tolist()
print('Number of assignment groups are: ' ,len(assignment_group_list))

#### How many configuration items are present in Preprocessed Data

In [ ]:
configuration_items_list = df['Configuration item'].tolist()

#### How many types of priority are present in Preprocessed Data

In [ ]:
# priority_list = df['Priority'].tolist()
priority_list =['2 - Medium','3 - Low','1 - High']

#### Created function to get average resolution time

In [ ]:
def mean_resolution_time_hr(df,assignment_group,configuration_item,priority_):
    
    hrs=3600
    data_df = df[(df['Assignment group']==assignment_group)&(df['Configuration item']==configuration_item)]
    data_df= data_df[data_df['Priority']==priority_]
    data_df  = data_df['Business resolve time'].mean()
    mean_time = (data_df/hrs)  
    return mean_time
    

In [ ]:
#assingment_group='Messaging'

In [ ]:
mean_resolution_time_hr

In [ ]:
result = pd.DataFrame(columns=['Assignment','Configuration','priority','Average_Resolution_Time_Hour'])

#### Creating the configuration item based on assignment groups in dictionary format

In [ ]:
def configuration_items(df,assignment_group):
    
    data = df[(df['Assignment group']==assignment_group)]
    data = data['Configuration item'].unique().tolist()
    return data    

In [ ]:
dict_assignment_configuration = {}
for assignment_group_name in assignment_group_list:
        configuration_items_list = configuration_items(df,assignment_group_name)
        dict_assignment_configuration.update({
        assignment_group_name: configuration_items_list
    })         

In [ ]:
# dict_assignment_configuration

 using the dictionary in terms of assignment group and its configuration item list and priority list,
looping in the for loop with mean_resolution_time_hr function and
 getting the average resolution time(hours)

In [ ]:
 for i, (dict_keys,dict_values) in enumerate(dict_assignment_configuration.items()):
    for configuration_item in dict_values:
        for priority in priority_list:
            values=(mean_resolution_time_hr(df,dict_keys,configuration_item,priority))
            mapping_values_with_result=[dict_keys,configuration_item,priority,values]
            result.loc[len(result)]=mapping_values_with_result          

In [ ]:
#mapping_values_with_result

#### output of the Average resolution_time(hrs)

In [ ]:
result

In [ ]:
result['Average_Resolution_Time_Hour']= result['Average_Resolution_Time_Hour'].round(2)

In [ ]:
result["Average_Resolution_Time_Hour"].fillna("Null", inplace= True)
result

#### saving the result in csv format

In [ ]:
result.to_csv('resolution_lookup.csv',index=False)